In [26]:
import rasterio as rio
import rasterio.mask
from rasterio.features import rasterize
import numpy as np
import geopandas as gpd
import xarray as xr

In [30]:
cyano_geopackage = gpd.read_file(r'c:\Kodning\data\satvistest\dataStructureTest\cyano_daymap.gpkg')

In [31]:
cyano_geopackage

,from_file,class,geometry
0,cyano_daymap_20020601.shp,1,"POLYGON ((190261 7342123, 206261 7342123, 2062..."
1,cyano_daymap_20020601.shp,1,"POLYGON ((377261 7339123, 379261 7339123, 3792..."
2,cyano_daymap_20020601.shp,1,"POLYGON ((182261 7324123, 185261 7324123, 1852..."
3,cyano_daymap_20020601.shp,1,"POLYGON ((1142261 7321123, 1144261 7321123, 11..."
4,cyano_daymap_20020601.shp,1,"POLYGON ((624261 7335123, 632261 7335123, 6322..."
...,...,...,...
1264132,cyano_daymap_20230926.shp,1,"POLYGON ((337261 5958123, 339261 5958123, 3392..."
1264133,cyano_daymap_20230926.shp,1,"POLYGON ((861261 5958123, 864261 5958123, 8642..."
1264134,cyano_daymap_20230926.shp,1,"POLYGON ((327261 5957123, 329261 5957123, 3292..."
1264135,cyano_daymap_20230926.shp,1,"POLYGON ((505261 6002123, 510261 6002123, 5102..."


In [34]:
xr.DataArray(cyano_geopackage)

<xarray.DataArray (dim_0: 1264137, dim_1: 3)> Size: 30MB
array([['cyano_daymap_20020601.shp', 1,
        <POLYGON ((190261 7342123, 206261 7342123, 206261 7339123, 205261 7339123, 2...>],
       ['cyano_daymap_20020601.shp', 1,
        <POLYGON ((377261 7339123, 379261 7339123, 379261 7338123, 381261 7338123, 3...>],
       ['cyano_daymap_20020601.shp', 1,
        <POLYGON ((182261 7324123, 185261 7324123, 185261 7323123, 184261 7323123, 1...>],
       ...,
       ['cyano_daymap_20230926.shp', 1,
        <POLYGON ((327261 5957123, 329261 5957123, 329261 5955123, 330261 5955123, 3...>],
       ['cyano_daymap_20230926.shp', 1,
        <POLYGON ((505261 6002123, 510261 6002123, 510261 6001123, 511261 6001123, 5...>],
       ['cyano_daymap_20230926.shp', 1,
        <POLYGON ((977261 7354123, 1350261 7354123, 1350261 6713123, 1349261 6713123...>]],
      dtype=object)
Coordinates:
  * dim_0    (dim_0) int64 10MB 0 1 2 3 4 ... 1264133 1264134 1264135 1264136
  * dim_1    (dim_1) object 24B 'from_file' 'class' 'geometry'

# Bottenviken raster mask

In [ ]:
basin_mapping_SVAR = {
    1: 'Bottenviken',
    2: 'Norra Kvarken',
    3: 'Bottenhavet',
    4: 'Ålands hav',
    5: 'Skärgårdshavet',
    6: 'Finska viken',
    7: 'Norra Gotlandshavet',
    8: 'Västra Gotlandshavet',
    9: 'Östra Gotlandshavet',
    10: 'Rigabukten',
    11: 'Gdanskbukten',
    12: 'Bornholmshavet & Hanöbukten',
    13: 'Arkonahavet & Södra Öresund',
    14: 'Bälthavet',
    15: 'Öresund',
    16: 'Kattegatt',
    17: 'Skagerrak',
}

In [3]:
basins = gpd.read_file("C:\\Kodning\\data\\satvistest\\basins_SVAR_2016.gpkg")

selected_basins = [3, 4, 5]
boolean_filter = basins['BASIN_NR'].isin(selected_basins)
basins = basins.loc[boolean_filter].set_index('BASIN_NR')
basins

,basin_name,geometry
BASIN_NR,,
3,Bottenhavet,"POLYGON ((620922.743 6729219.402, 620757.133 6..."
4,Ålands hav,"POLYGON ((709486.835 6669336.804, 709413.026 6..."
5,Skärgårdshavet,"POLYGON ((941642.96 6656019.222, 941484.624 66..."


In [17]:
basins['geometry'][3]

In [18]:
shapes = [basins['geometry'][3], basins['geometry'][4], basins['geometry'][5]]

In [36]:
with rio.open("c:\\Arbetsmapp\\BAWS\\Årsrapport 2023\\Data_2002-2023\\aggregate_archive\\all_years\\aggregation_2017-2023.tiff") as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [25]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rio.open("c:\\Arbetsmapp\\BAWS\\Årsrapport 2023\\Data_2002-2023\\aggregate_archive\\all_years\\aggregation_2017-2023.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
rst = rio.open(fid)
array = rst.read()
array = array[0].astype(int)
if include_meta:
    meta = rst.meta.copy()
    meta.update(compress='lzw')
    return array, meta
else:
    return array

In [ ]:
mask, meta = raster_reader(r'C:\Kodning\BAWS-vis\raster_landmask_baws1000_sweref99tm.tiff', include_meta=True)

In [ ]:
files = generate_filepaths(
    r'C:\Arbetsmapp\BAWS\Årsrapport 2023\Data_test\baws_rasterize\prior_years',
    pattern='cyano_daymap',
    endswith='.tiff',
)

In [ ]:
for fid in files:
    print(fid)
    cyano_day = raster_reader(fid)

    indexer = np.logical_and(
        np.logical_or(cyano_day == 2, cyano_day == 3),
        mask == 0
    )

    cyano_day = np.where(indexer, 0, cyano_day)

    with rio.open(fid, 'w', **meta) as dst:
        dst.write(cyano_day.astype(rio.uint8), 1)
    dst.close()